In [ ]:
print('[OPT] Data Cleansing - cleanse_portdistance')

In [1]:
json_request={
    "parameter": {
        "data": {
            "idConfRoute": "2b3aac5f-f786-4430-b37f-f75ccd247960",
            "forbiddenPort": "80a4b48e-2bbb-42f8-b0f5-aed166f81eb3, bb5c997e-f3ac-4b41-92d1-23458aa5c1f1",
            "status": "RUNNING",
            "runningTime": 0,
            "accuracy": 0,
            "method": "string",
            "trial": 0,
            "routeDetail": [
            {
                "idConfRouteDetail": "ff0dba1c-6211-4b52-9cea-c7a49822e05a",
                "idShip": "f3b7a6b0-21a3-45e5-82dc-0eaaec39ede4",  
                "lastLocationPort": "05aa9d6c-6999-4f7a-b4d6-7dddca9db337",
                "originPort": "19887a24-9011-425f-bc0c-5f5cd512ba5a",
                "destinationPort": "",
                "mustVisitPort": "",
                "minPort": 2,
                "maxPort": 10,
                "minComDays": 1,
                "maxComDays": 14,
                "regionRule": "NOT_SPECIFIED",
                "roundTrip": 1
            },
            {
                "idConfRouteDetail": "d1339ee1-9472-4d34-a722-000b9edc4779",
                "idShip": "d4787f56-56d0-45ae-8822-ce55b720357a",  
                "lastLocationPort": "19887a24-9011-425f-bc0c-5f5cd512ba5a",
                "originPort": "",
                "destinationPort": "3c42cdbb-1dd7-4214-99d3-ffabc2c2a93d",
                "mustVisitPort": "6a01e7c5-05e6-4ab7-a706-f0936f32a837, 3c42cdbb-1dd7-4214-99d3-ffabc2c2a93d",
                "minPort": 3,
                "maxPort": 50,
                "minComDays": 1,
                "maxComDays": 28,
                "regionRule": "NOT_SPECIFIED",
                "roundTrip": 1
            }]
        }
    }
}

In [2]:
from geopy.distance import geodesic as gd


In [3]:
import pandas as pd
import numpy as np
import time
from typing import List,Callable,Tuple
from collections import namedtuple
from functools import partial
import import_ipynb
from data_preparation import *
from general_function import *
import itertools

input_request = input_df(json_request)

idConf = input_request[0]
forbidden = input_request[1]
df_input = input_request[2]

##EXECUTE GLOBAL VARIABLES

#master port, exclude forbiddenport
df_port_all = retrieve_port('001')
df_port = df_port_all[~df_port_all['id'].isin([forbidden])]

#master ship, include input ship only
df_ship_all = retrieve_ship('001')
df_ship = df_ship_all[df_ship_all['id'].isin(df_input.idShip)]
df_ship=df_ship.fillna(0)
ship_input=pd.merge(df_ship,df_input, how='inner', left_on=['id'], right_on=['idShip'])

#master port distance
df_portdistance = retrieve_portdistance()



importing Jupyter notebook from data_preparation.ipynb
importing Jupyter notebook from general_function.ipynb


In [ ]:
portlist = list(df_port.id)
portlist.append('111')
od_pair = pd.DataFrame((itertools.permutations(portlist, 2)), columns=['id_origin', 'id_destination'])

df_join = pd.merge(df_portdistance, od_pair, how='right', left_on=('id_origin', 'id_destination'), right_on=('id_origin', 'id_destination'))
notfound = df_join[df_join['nautical'].isna()]

In [ ]:
df2 = df_port[df_port['portno'] == 10]
df2 = df2.reset_index()
df2.drop('portno', axis=1, inplace=True)

df2['id'], df2['portcode'], df2['name'], df2['code'], df2['lat'], df2['long'], df2['hightide'], df2['lowtide'], df2['length'] = '111', 'AZZZ', 'DUMMY', '99001', '-6.1312617', '106.6409313', '3.1', '4.1', '99'

In [ ]:
df_port = pd.concat([df_port, df2])
df_port

port1 = df_port.loc[:, ['id','lat', 'long']]
port3 = notfound.loc[:, ['id_origin','id_destination']]

In [ ]:
port2 = port1
port1 = port1.rename(columns={'id': 'id_origin', 'lat': 'lat_origin', 'long': 'long_origin'})
port2 = port2.rename(columns={'id': 'id_destination', 'lat': 'lat_destination', 'long': 'long_destination'})

In [ ]:
# Change Typedate
port3['id_origin'] = port3['id_origin'].astype(str)
port1['id_origin'] = port1['id_origin'].astype(str)

# Inner Join
inner_join = pd.merge(port3, 
                      port1, 
                      on ='id_origin', 
                      how ='inner')

In [ ]:
# Change Typedata
inner_join['id_destination'] = inner_join['id_destination'].astype(str)
port2['id_destination'] = port2['id_destination'].astype(str)

# Inner Join
merge_data = pd.merge(inner_join,
                      port2,
                      on ='id_destination', 
                      how ='outer')

In [ ]:
# Change Typedata
cleandf = merge_data

cleandf['lat_origin'] = cleandf['lat_origin'].astype(float)
cleandf['long_origin'] = cleandf['long_origin'].astype(float)

cleandf['lat_destination'] = cleandf['lat_destination'].astype(float)
cleandf['long_destination'] = cleandf['long_destination'].astype(float)

In [ ]:
# Calculate from latitude and longtitude data convert to miles (use other library)
nautical_data2 = []
for index, row in cleandf.iterrows():
    ori = (row['lat_origin'],row['long_origin'])
    des = (row['lat_destination'],row['long_destination'])
    rs = gd(ori, des).miles
    rs = round(rs)
    nautical_data2.append(rs)
    
# Add new columns    
cleandf['nautical'] = nautical_data2
cleandf['commercial'] = nautical_data2

In [4]:
def cleansing_portdistance(df_port, df_portdistance) :
    #global df_portdistance
    portlist = list(df_port.id)
    portlist.append('111')
    od_pair = pd.DataFrame((itertools.permutations(portlist, 2)), columns=['id_origin', 'id_destination'])

    # Permutation (id_port)
    df_join = pd.merge(df_portdistance, od_pair, how='right', left_on=('id_origin', 'id_destination'), right_on=('id_origin', 'id_destination'))
    notfound = df_join[df_join['nautical'].isna()]

    # Add data dummy
    dum = df_port[df_port['portno'] == 10]
    dum = dum.reset_index()
    dum.drop('portno', axis=1, inplace=True)
    
    # Update data dummy
    dum['id'], dum['portcode'], dum['name'], dum['code'], dum['lat'], dum['long'], dum['hightide'], dum['lowtide'], dum['length'] = '111', 'AZZZ', 'DUMMY', '99001', '-6.1312617', '106.6409313', '3.1', '4.1', '99'
    
    df_port = pd.concat([df_port, dum])
    df_ori = df_port.loc[:, ['id','lat', 'long']]
    nf = notfound.loc[:, ['id_origin','id_destination']]
    df_des = df_ori

    # Change name columns
    df_ori = df_ori.rename(columns={'id': 'id_origin', 'lat': 'lat_origin', 'long': 'long_origin'})
    df_des = df_des.rename(columns={'id': 'id_destination', 'lat': 'lat_destination', 'long': 'long_destination'})
    
    # Change Typedate
    nf['id_origin'] = nf['id_origin'].astype(str)
    df_ori['id_origin'] = df_ori['id_origin'].astype(str)

    # Inner Join
    df = pd.merge(nf, 
                df_ori, 
                on ='id_origin', 
                how ='inner')
    
    # Change Typedata
    df['id_destination'] = df['id_destination'].astype(str)
    df_des['id_destination'] = df_des['id_destination'].astype(str)

    # Inner Join
    df = pd.merge(df,
                    df_des,
                    on ='id_destination', 
                    how ='inner')

    # Change Typedata
    df['lat_origin'] = df['lat_origin'].astype(float)
    df['long_origin'] = df['long_origin'].astype(float)
    df['lat_destination'] = df['lat_destination'].astype(float)
    df['long_destination'] = df['long_destination'].astype(float)

    # Calculate from latitude and longtitude data convert to miles (use other library)
    nautical_data = []
    for index, row in df.iterrows():
        ori = (row['lat_origin'],row['long_origin'])
        des = (row['lat_destination'],row['long_destination'])
        rs = gd(ori, des).miles
        rs = round(rs)
        nautical_data.append(rs)
        
    # Add new columns    
    df['nautical'] = nautical_data
    df['commercial'] = nautical_data
    df['origin'] = 99999
    df['destinaton'] = 99999
    
    return df

In [7]:
cleansing_portdistance(df_port, df_portdistance)

,id_origin,id_destination,lat_origin,long_origin,lat_destination,long_destination,nautical,commercial,origin,destinaton
0,cc0756e0-437b-4403-b4c9-14d4bc895620,111,1.050458,120.800080,-6.131262,106.640931,1095,1095,99999,99999
1,ddac0441-65ff-4960-b993-87c73ebc2d85,111,-1.629990,105.733000,-6.131262,106.640931,316,316,99999,99999
2,3e8f594a-bf5a-43ff-aeb5-2bdcc285d0dd,111,-2.539418,112.964690,-6.131262,106.640931,501,501,99999,99999
3,a5e44d1c-a5e0-44de-831f-69aeb13500f7,111,-3.432063,116.009030,-6.131262,106.640931,672,672,99999,99999
4,5aec424d-b383-4ed5-9dc4-cb14e75ade0a,111,-8.730020,116.077140,-6.131262,106.640931,671,671,99999,99999
...,...,...,...,...,...,...,...,...,...,...
201,111,ffe0f370-0b31-4d63-9e7f-10425a8dc93b,-6.131262,106.640931,3.000583,107.744780,632,632,99999,99999
202,111,55c79ac1-55fa-4726-b3f8-f3264fbb674d,-6.131262,106.640931,-8.614589,122.219864,1082,1082,99999,99999
203,111,85c90424-c458-4045-8719-459d9074f3cf,-6.131262,106.640931,-8.221984,124.512360,1235,1235,99999,99999
204,111,0b4c8502-405b-4950-9cdc-049b94c16047,-6.131262,106.640931,-9.391875,119.220354,891,891,99999,99999
